<a href="https://colab.research.google.com/github/rajan-sharma-in/GDG/blob/main/AI_Buddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Slide 0: Install pinned versions that WORK together.
# Why: Avoids random Colab dependency conflicts (huggingface-hub / transformers / gradio).

%pip -q install -U faiss-cpu gradio pymupdf google-genai

# IMPORTANT: pin huggingface-hub < 1.0 (required by transformers/tokenizers)
%pip -q uninstall -y huggingface-hub
%pip -q install -U "huggingface-hub==0.35.0"

%pip -q install -U \
  "transformers==4.45.2" \
  "sentence-transformers==3.1.1" \
  "tokenizers==0.20.1" \
  "safetensors==0.4.5"

# Pillow pin to avoid occasional PIL typing import errors
%pip -q install -U "pillow==10.4.0"


In [ ]:
# Slide 1: Imports + global settings
import re
from pathlib import Path

import numpy as np
import pandas as pd
import fitz  # PyMuPDF
import faiss

from sentence_transformers import SentenceTransformer

print("✅ Imports OK")

# --- Settings students can change ---
EMBED_MODEL_NAME = "all-MiniLM-L6-v2"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 180
TOP_K_DEFAULT = 3

# Embedding model (DON'T name it 'model' to avoid confusion with Gemini)
embed_model = SentenceTransformer(EMBED_MODEL_NAME)
print("✅ Embedding model loaded:", EMBED_MODEL_NAME)


✅ Imports OK
✅ Embedding model loaded: all-MiniLM-L6-v2


In [ ]:
# Slide 2: Choose ONE PDF from Google Drive (recommended) or local storage
# Why: Students can keep their books in Drive and reuse them across sessions.

from pathlib import Path

USE_DRIVE = True  # <-- set False if you don't want Google Drive

if USE_DRIVE:
    from google.colab import drive
    drive.mount("/content/drive")

    PDF_DIR = Path("/content/drive/MyDrive/Colab_PDFs")
    CACHE_DIR = Path("/content/drive/MyDrive/Colab_PDFs/index_cache")
else:
    PDF_DIR = Path("/content/pdfs")
    CACHE_DIR = Path("/content/index_cache")

PDF_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

def list_pdfs(max_show=30):
    pdfs = sorted(PDF_DIR.glob("*.pdf"))
    print(f"📁 PDF folder: {PDF_DIR}")
    print(f"✅ Found {len(pdfs)} PDF(s).")
    for p in pdfs[:max_show]:
        print(" -", p.name)
    if len(pdfs) > max_show:
        print(f" ... (showing first {max_show})")
    return pdfs

pdf_paths = list_pdfs()

assert len(pdf_paths) > 0, (
    f"No PDFs found in {PDF_DIR}. "
    "Upload a PDF to this folder and re-run this cell."
)

# Pick the PDF:
# Option A (simple): always pick the first PDF
PDF_PATH = pdf_paths[0]

# Option B (manual selection): set a file name explicitly
# PDF_TO_USE = "your_book.pdf"
# PDF_PATH = PDF_DIR / PDF_TO_USE

assert PDF_PATH.exists(), f"PDF not found: {PDF_PATH}"
print("✅ Using PDF:", PDF_PATH)
print("✅ Cache folder:", CACHE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 PDF folder: /content/drive/MyDrive/Colab_PDFs
✅ Found 1 PDF(s).
 - 16_EBOOK-7th_ed_software_engineering_a_practitioners_approach_by_roger_s._pressman_.pdf
✅ Using PDF: /content/drive/MyDrive/Colab_PDFs/16_EBOOK-7th_ed_software_engineering_a_practitioners_approach_by_roger_s._pressman_.pdf
✅ Cache folder: /content/drive/MyDrive/Colab_PDFs/index_cache


In [ ]:
# Slide 3: Extract text from the PDF page-by-page.
# Why: We keep page_number so we can cite the exact page in results.

def extract_pages(pdf_path: Path):
    doc = fitz.open(pdf_path)
    rows = []
    for i in range(doc.page_count):
        page = doc.load_page(i)
        text = page.get_text("text") or ""
        rows.append({
            "file_name": pdf_path.name,
            "page_number": i + 1,
            "text": text
        })
    return pd.DataFrame(rows), doc.page_count

pages_df, total_pages = extract_pages(PDF_PATH)
print("✅ Pages extracted:", len(pages_df), "| Total pages:", total_pages)
pages_df.head(3)


✅ Pages extracted: 930 | Total pages: 930


,file_name,page_number,text
0,16_EBOOK-7th_ed_software_engineering_a_practit...,1,Software Engineering\nA Practitioner’s Approac...
1,16_EBOOK-7th_ed_software_engineering_a_practit...,2,Software Engineering\nA\nP R A C T I T I O N E...
2,16_EBOOK-7th_ed_software_engineering_a_practit...,3,


In [ ]:
# Slide 4: Split text into overlapping chunks.
# Why: Search works better on smaller chunks than entire pages/books.

def clean_text(t: str) -> str:
    return re.sub(r"\s+", " ", (t or "")).strip()

def make_chunks(pages_df, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    chunks = []
    for row in pages_df.itertuples(index=False):
        text = clean_text(row.text)
        if not text:
            continue

        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            chunks.append({
                "file_name": row.file_name,
                "page_number": int(row.page_number),
                "chunk_text": chunk_text
            })
            start = end - overlap

    return pd.DataFrame(chunks)

chunks_df = make_chunks(pages_df)
print("✅ Chunks created:", len(chunks_df))
chunks_df.head(3)


✅ Chunks created: 3502


,file_name,page_number,chunk_text
0,16_EBOOK-7th_ed_software_engineering_a_practit...,1,Software Engineering A Practitioner’s Approach...
1,16_EBOOK-7th_ed_software_engineering_a_practit...,2,Software Engineering A P R A C T I T I O N E R...
2,16_EBOOK-7th_ed_software_engineering_a_practit...,4,Software Engineering A P R A C T I T I O N E R...


In [ ]:
# Slide 5: Build embeddings and a FAISS index (cosine similarity).
# Also cache to Drive/local so you don't rebuild every time.
import json
INDEX_PATH = CACHE_DIR / "faiss.index"
META_PATH  = CACHE_DIR / "chunks.parquet"
EMB_PATH   = CACHE_DIR / "embeddings.npy"
MANIFEST_PATH = CACHE_DIR / "manifest.json"

def build_manifest(pdf_path: Path):
    stat = pdf_path.stat()
    return {
        "file_name": pdf_path.name,
        "mtime": stat.st_mtime,
        "size": stat.st_size,
        "chunk_size": CHUNK_SIZE,
        "overlap": CHUNK_OVERLAP,
        "embed_model": EMBED_MODEL_NAME,
    }

def load_cache_if_valid(pdf_path: Path):
    # json is needed here (and already imported in Slide 1)
    if not (INDEX_PATH.exists() and META_PATH.exists() and EMB_PATH.exists() and MANIFEST_PATH.exists()):
        return None

    with open(MANIFEST_PATH, "r") as f:
        saved = json.load(f)

    current = build_manifest(pdf_path)
    if saved != current:
        return None

    try:
        df = pd.read_parquet(META_PATH)
        emb = np.load(EMB_PATH).astype("float32")
        idx = faiss.read_index(str(INDEX_PATH))
        print("Loaded cached FAISS index + embeddings.")
        return df, emb, idx
    except Exception as e:
        print("Cache load failed:", e)
        return None

def save_cache(pdf_path: Path, chunks_df: pd.DataFrame, emb_matrix: np.ndarray, idx):
    chunks_df.to_parquet(META_PATH, index=False)
    np.save(EMB_PATH, emb_matrix)
    faiss.write_index(idx, str(INDEX_PATH))
    with open(MANIFEST_PATH, "w") as f:
        json.dump(build_manifest(pdf_path), f)
    print("Cache saved in:", CACHE_DIR)

def build_index(chunks_df: pd.DataFrame):
    texts = chunks_df["chunk_text"].tolist()

    emb = embed_model.encode(
        texts,
        batch_size=64,
        convert_to_numpy=True,
        show_progress_bar=True
    ).astype("float32")

    # Normalize so dot product == cosine similarity
    faiss.normalize_L2(emb)

    dim = emb.shape[1]
    idx = faiss.IndexFlatIP(dim)
    idx.add(emb)

    return emb, idx

cached = load_cache_if_valid(PDF_PATH)
if cached:
    chunks_df, emb_matrix, index = cached
else:
    emb_matrix, index = build_index(chunks_df)
    save_cache(PDF_PATH, chunks_df, emb_matrix, index)

print("FAISS ready | vectors:", index.ntotal, "| dim:", emb_matrix.shape[1])


Batches:   0%|          | 0/55 [00:00<?, ?it/s]

Cache saved in: /content/drive/MyDrive/Colab_PDFs/index_cache
FAISS ready | vectors: 3502 | dim: 384


In [ ]:
# Slide 6: Retrieval (Top-K chunks)

def confidence_label(score: float) -> str:
    if score >= 0.65: return "High"
    if score >= 0.40: return "Medium"
    return "Low"

def retrieve_chunks(query: str, k: int = 3):
    query = clean_text(query)
    if not query:
        return []

    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)

    k = min(int(k), index.ntotal)
    scores, ids = index.search(q_emb, k)

    items = []
    for score, idx_id in zip(scores[0], ids[0]):
        row = chunks_df.iloc[int(idx_id)]
        items.append({
            "score": float(score),
            "confidence": confidence_label(float(score)),
            "file_name": row["file_name"],
            "page_number": int(row["page_number"]),
            "chunk_text": row["chunk_text"]
        })
    return items

def format_transparency(items):
    lines = []
    for it in items:
        cite = f"[{it['file_name']} p.{it['page_number']}]"
        preview = it["chunk_text"][:240].replace("\n", " ")
        lines.append(f"{it['score']:.3f} | {it['confidence']} | {cite}\n{preview}...")
    return "\n\n".join(lines)

# Smoke test
question = "what is software testing strategies"
items = retrieve_chunks(question, k=TOP_K_DEFAULT)

raw_context = "\n\n".join([f"[{it['file_name']} p.{it['page_number']}]\n{it['chunk_text']}" for it in items])
transparency = format_transparency(items)
confidence = f"{items[0]['confidence']} (top score {items[0]['score']:.3f})" if items else "Low"

print("Question:", question)
print("Confidence:", confidence)
print("\nTransparency (top results):\n")
print(transparency[:1200], "...")


Question: what is software testing strategies
Confidence: High (top score 0.812)

Transparency (top results):

0.812 | High | [16_EBOOK-7th_ed_software_engineering_a_practitioners_approach_by_roger_s._pressman_.pdf p.478]
A strategy for software testing provides a road map that describes the steps to be conducted as part of testing, when these steps are planned and then undertaken, and how much effort, time, and resources will be required. Therefore, any tes...

0.789 | High | [16_EBOOK-7th_ed_software_engineering_a_practitioners_approach_by_roger_s._pressman_.pdf p.478]
17 SOFTWARE TESTING STRATEGIES What is it? Software is tested to uncover errors that were made inad- vertently as it was designed and constructed. But how do you conduct the tests? Should you develop a formal plan for your tests? Should you...

0.741 | High | [16_EBOOK-7th_ed_software_engineering_a_practitioners_approach_by_roger_s._pressman_.pdf p.479]
450 PART THREE QUALITY MANAGEMENT integration testing . . . . . . 

In [ ]:
# Slide 7: Gemini synthesis (context-only).
# Why: Gemini converts retrieved chunks into a clean answer without hallucination.

from google import genai
from google.genai import types
from google.colab import userdata

_GENAI_CLIENT = None

def get_genai_client():
    global _GENAI_CLIENT
    if _GENAI_CLIENT is not None:
        return _GENAI_CLIENT

    API_KEY = userdata.get("GOOGLE_API_KEY")
    assert API_KEY, "Set GOOGLE_API_KEY in Colab Secrets"

    _GENAI_CLIENT = genai.Client(
        api_key=API_KEY,
        http_options=types.HttpOptions(
            api_version="v1",
            timeout=90_000,  # ms
            retry_options=types.HttpRetryOptions(
                attempts=4,
                initial_delay=1.0,
                max_delay=15.0,
            ),
        ),
    )
    return _GENAI_CLIENT

def clip(text, n=12000):
    text = "" if text is None else str(text)
    return text[:n] + ("...[trimmed]" if len(text) > n else "")

def gemini_context_only_answer(question: str, context: str, model="gemini-2.5-flash"):
    client = get_genai_client()

    prompt = (
        "Answer the question using ONLY the context below.\n"
        "If the context does not contain the answer, say: Not found in the provided context.\n\n"
        f"Question:\n{question}\n\n"
        f"Context:\n{clip(context)}"
    )

    resp = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            max_output_tokens=512,
            temperature=0.2,
        ),
    )
    return getattr(resp, "text", "") or ""

# Demo Gemini output
llm_answer = gemini_context_only_answer(question, raw_context)
print("Gemini Answer:\n")
print(llm_answer)


Gemini Answer:

A strategy for software testing provides a road map that describes the steps to be conducted as part of testing,


In [ ]:
# Slide 8: Pipeline (retrieval + optional Gemini)
# Returns everything needed for UI and debugging.

def pipeline(question: str, top_k: int = 3, use_gemini: bool = True):
    items = retrieve_chunks(question, k=int(top_k))
    if not items:
        return "No relevant text found in the PDF.", "Low", "", "No retrieval", ""

    conf = f"{items[0]['confidence']} (top score {items[0]['score']:.3f})"
    raw_ctx = "\n\n".join([f"[{it['file_name']} p.{it['page_number']}]\n{it['chunk_text']}" for it in items])
    trans = format_transparency(items)

    if not use_gemini:
        answer = "I found this in the PDF:\n\n" + raw_ctx
        return answer, conf, trans, "Direct retrieval (no LLM)", raw_ctx

    try:
        answer = gemini_context_only_answer(question, raw_ctx)
        return answer, conf, trans, "RAG with Gemini (context-only)", raw_ctx
    except Exception as e:
        answer = f"[Gemini error: {e}]\n\nI found this in the PDF:\n\n{raw_ctx}"
        return answer, conf, trans, "Fallback to retrieval", raw_ctx

# Quick test
ans, conf, trans, route, raw_ctx = pipeline("what is software testing strategies", top_k=3, use_gemini=True)
print(route)
print(conf)
print(ans[:700], "...")


RAG with Gemini (context-only)
High (top score 0.812)
A strategy for software testing provides a road map that describes the steps to be conducted as part of ...


In [ ]:
# Slide 9: Gradio UI

import gradio as gr

def ui_query(question, top_k, use_gemini):
    ans, conf, trans, route, raw_ctx = pipeline(
        question, top_k=int(top_k), use_gemini=bool(use_gemini)
    )
    return ans, conf, route, raw_ctx, trans

with gr.Blocks(title="One-PDF RAG Demo") as demo:
    gr.Markdown("# One-PDF RAG Demo (FAISS + SentenceTransformers + Gemini)")
    gr.Markdown(f"Current PDF: `{PDF_PATH.name}` | Pages: `{total_pages}`")

    with gr.Row():
        with gr.Column(scale=1):
            top_k = gr.Slider(1, 10, value=3, step=1, label="Top-K Chunks")
            use_gemini = gr.Checkbox(label="Use Gemini (context-only)", value=True)

        with gr.Column(scale=2):
            question = gr.Textbox(label="Question", placeholder="e.g., What is software testing strategies?")
            ask = gr.Button("Ask")

    answer = gr.Textbox(label="Answer", lines=10)
    confidence = gr.Textbox(label="Confidence")
    route = gr.Textbox(label="Route")
    raw_context = gr.Textbox(label="Raw Retrieved Context", lines=10)
    transparency = gr.Textbox(label="Retrieval Transparency", lines=10)

    ask.click(ui_query, inputs=[question, top_k, use_gemini],
              outputs=[answer, confidence, route, raw_context, transparency])
    question.submit(ui_query, inputs=[question, top_k, use_gemini],
                    outputs=[answer, confidence, route, raw_context, transparency])

demo.queue(default_concurrency_limit=1)
demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3c53d3c893f765f788.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
